In [1]:
import gc
gc.collect()

200

In [2]:
# 各ライブラリのインストール
# Readme参照

In [3]:
!nvidia-smi

Wed Jul  2 14:10:59 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.05    Driver Version: 510.73.05    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A4000    On   | 00000000:19:00.0 Off |                  Off |
| 41%   42C    P8    15W / 140W |      5MiB / 16376MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A4000    On   | 00000000:1A:00.0 Off |                  Off |
| 41%   

In [4]:
!pip show torch torchvision

Name: torch
Version: 2.7.1+cu118
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /data2/tomoshi0514/anaconda3/envs/AdafaceTest/lib/python3.12/site-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu11, nvidia-cuda-cupti-cu11, nvidia-cuda-nvrtc-cu11, nvidia-cuda-runtime-cu11, nvidia-cudnn-cu11, nvidia-cufft-cu11, nvidia-curand-cu11, nvidia-cusolver-cu11, nvidia-cusparse-cu11, nvidia-nccl-cu11, nvidia-nvtx-cu11, setuptools, sympy, triton, typing-extensions
Required-by: pytorch-lightning, torchaudio, torchmetrics, torchvision
---
Name: torchvision
Version: 0.22.1+cu118
Summary: image and video datasets and models for torch deep learning
Home-page: https://github.com/pytorch/vision
Author: PyTorch Core Team
Author-email: soumith@pytorch.org
License: BSD
Location: /data2/tomoshi0514/anaconda3/envs/AdafaceTest/lib/pyth

# CONFIG

In [5]:
import os

class CONFIG:
    
    # デバックモード
    is_debug_mode = True
    max_class_num = 5

    # 学習対象のファイル形式
    use_mxrecord = True
    # 検証ファイルを作成するか
    make_validation_memfiles = False

    # データセットの指定
    is_original_dataset = False
    dataset_name = 'faces_webface_112x112' if not is_original_dataset else 'original'
    dataset_dir = os.path.join("data", dataset_name)
    

In [6]:
import gdown
import zipfile

if not os.path.isdir(CONFIG.dataset_dir):
    if CONFIG.dataset_name == 'faces_umd':
        #UMD-facesをダウンロード
        file_id = '1azhEHoJjVmifuzBVKJwl-sDbLZ-Wzp4O'
        gdown.download(f"https://drive.google.com/uc?id={file_id}", f"{CONFIG.dataset_name}.zip", quiet=False)
        #zip解凍
        with zipfile.ZipFile(f"{CONFIG.dataset_name}.zip", 'r') as zip_ref:
            zip_ref.extractall("data")  # 任意の展開先フォルダ
    if CONFIG.dataset_name == 'faces_webface_112x112':
        # CASIA-webfaceをダウンロード
        file_id = '1KxNCrXzln0lal3N4JiYl9cFOIhT78y1l'
        gdown.download(f"https://drive.google.com/uc?id={file_id}", f"{CONFIG.dataset_name}.zip", quiet=False)
        #zip解凍
        with zipfile.ZipFile(f"{CONFIG.dataset_name}.zip", 'r') as zip_ref:
            zip_ref.extractall("data")  # 任意の展開先フォルダ

    if CONFIG.dataset_name == 'original':
        pass


# rec file convert to image files

In [7]:
# recordIO形式をImageFile形式に変換する（dataset_dir直下にimgsディレクトリが生成される）
if not os.path.exists(f"{CONFIG.dataset_dir}/imgs") and not CONFIG.is_original_dataset and not CONFIG.use_mxrecord: # InsightFaceDataset
    !python AdaFace/convert.py --rec_path {CONFIG.dataset_dir} --make_image_files 
elif not os.path.exists(f"{CONFIG.dataset_dir}/train.rec") and CONFIG.is_original_dataset and not CONFIG.use_mxrecord : # OriginalFaceDataset
    pass
else:
    pass

In [8]:
data_root = os.path.dirname(CONFIG.dataset_dir) 
dataset_dir = CONFIG.dataset_dir
dataset_img_dir = os.path.join(CONFIG.dataset_dir, "imgs")

print(f" data_root: {data_root}\n dataset_dir: {dataset_dir}\n dataset_img_dir:{dataset_img_dir}") 

 data_root: data
 dataset_dir: data/faces_webface_112x112
 dataset_img_dir:data/faces_webface_112x112/imgs


In [9]:
import shutil

debug_dataset_dir = os.path.join(data_root, "debug_dataset")
debug_img_dir = os.path.join(debug_dataset_dir, "imgs") 

# デバッグモードの場合、max_class_num分dataset_dirのimgsファイルをコピー、対象データディレクトリをdebug_datasetディレクトリに変更
if CONFIG.is_debug_mode:

    # 既存フォルダをクリア
    if os.path.isdir(debug_dataset_dir):
        os.remove(f"{debug_dataset_dir}/train.idx") if os.path.exists(f"{debug_dataset_dir}/train.idx") else None
        os.remove(f"{debug_dataset_dir}/train.lst") if os.path.exists(f"{debug_dataset_dir}/train.lst") else None
        os.remove(f"{debug_dataset_dir}/train.rec") if os.path.exists(f"{debug_dataset_dir}/train.rec") else None
        shutil.rmtree(f"{debug_dataset_dir}/imgs") 
    os.makedirs(debug_dataset_dir, exist_ok=True)

    # クラス名ディレクトリをソートして先頭5つ取得
    class_dirs = sorted([
        d for d in os.listdir(dataset_img_dir)
        if os.path.isdir(os.path.join(dataset_img_dir, d))
    ])[:CONFIG.max_class_num]

    # 画像データをImageFolder形式としてコピー
    for cls in class_dirs:
        src = os.path.join(dataset_img_dir, cls)
        dst = os.path.join(debug_img_dir, cls)
        shutil.copytree(src, dst)
        # もしくは高速に済ませたいなら symlink:
        # os.symlink(src, dst, target_is_directory=True)

    # dataset_dirをdebug_dataset_dirに変更
    dataset_dir = debug_dataset_dir
    dataset_img_dir = debug_img_dir

    print("============== debug file is created ============")
    
else:
    print("==== you're using raw data (not using debug directory)")

print(f" data_root: {data_root}\n dataset_dir: {dataset_dir}\n dataset_img_dir:{dataset_img_dir}")

# recordIO形式で訓練する場合、.recに変換
if CONFIG.use_mxrecord:
    print("========== create new rec file =========")
    !python create_train_rec.py --img_dir {dataset_img_dir} --output_dir {dataset_dir}
    print(".rec .lst .bin is created")

dataset_dir_name = os.path.basename(dataset_dir)
dataset_img_dir_name = os.path.basename(dataset_img_dir)
print(f" data_root: {data_root}\n dataset_dir: {dataset_dir}\n dataset_img_dir:{dataset_img_dir}")
print(f"▶ Using dataset_path : {dataset_dir}")

============== debug file is created ============
 data_root: data
 dataset_dir: data/debug_dataset
 dataset_img_dir:data/debug_dataset/imgs
========== create new rec file =========
100%|███████████████████████████████████████| 473/473 [00:00<00:00, 3767.49it/s]
Wrote 473 images belonging to 5 classes
Record file : data/debug_dataset/train.rec
Index file  : data/debug_dataset/train.idx
List file   : data/debug_dataset/train.lst
.rec .lst .bin is created
 data_root: data
 dataset_dir: data/debug_dataset
 dataset_img_dir:data/debug_dataset/imgs
▶ Using dataset_path : data/debug_dataset


In [10]:
import os

# CASIA webfaceデータセットから対象.binファイルを事前にコピー済み
validation_dir = os.path.join(data_root, "validation_dataset")
val_dataset_dirs = ["agedb_30", "calfw", "cfp_ff", "cfp_fp", "cplfw", "lfw"]
validation_dir_name = os.path.basename(validation_dir)

if all(os.path.exists(f"{validation_dir}/{d}") for d in val_dataset_dirs):
    print("val dir is already exists")
else:
    print("some val data is not exists : convert .bin to new val data")
    # 検証ファイルを構築
    !python AdaFace/convert.py --rec_path {validation_dir} --make_validation_memfiles

some val data is not exists : convert .bin to new val data
loading bin 1000
loading bin 2000
loading bin 3000
loading bin 4000
loading bin 5000
loading bin 6000
loading bin 7000
loading bin 8000
loading bin 9000
loading bin 10000
loading bin 11000
loading bin 12000
(12000, 3, 112, 112)
loading bin 1000
loading bin 2000
loading bin 3000
loading bin 4000
loading bin 5000
loading bin 6000
loading bin 7000
loading bin 8000
loading bin 9000
loading bin 10000
loading bin 11000
loading bin 12000
loading bin 13000
loading bin 14000
(14000, 3, 112, 112)
loading bin 1000
loading bin 2000
loading bin 3000
loading bin 4000
loading bin 5000
loading bin 6000
loading bin 7000
loading bin 8000
loading bin 9000
loading bin 10000
loading bin 11000
loading bin 12000
(12000, 3, 112, 112)
loading bin 1000
loading bin 2000
loading bin 3000
loading bin 4000
loading bin 5000
loading bin 6000
loading bin 7000
loading bin 8000
loading bin 9000
loading bin 10000
loading bin 11000
loading bin 12000
(12000, 3, 112

# train

In [12]:
!python AdaFace/main.py --help

usage: main.py [--data_root DATA_ROOT] [--train_data_path TRAIN_DATA_PATH]
               [--val_data_path VAL_DATA_PATH] [--use_mxrecord]
               [--train_data_subset] [--swap_color_channel] [--prefix PREFIX]
               [--gpus GPUS] [--distributed_backend {dp,ddp,ddp2}]
               [--use_16bit] [--epochs N] [--seed SEED]
               [--batch_size BATCH_SIZE] [--lr LR]
               [--lr_milestones LR_MILESTONES] [--lr_gamma LR_GAMMA]
               [--num_workers NUM_WORKERS] [--fast_dev_run] [--evaluate]
               [--resume_from_checkpoint RESUME_FROM_CHECKPOINT]
               [--start_from_model_statedict START_FROM_MODEL_STATEDICT]
               [--use_wandb] [--custom_num_class CUSTOM_NUM_CLASS]
               [--arch ARCH] [--momentum M] [--weight_decay WEIGHT_DECAY]
               [--head HEAD] [--m M] [--h H] [--s S] [--t_alpha T_ALPHA]
               [--low_res_augmentation_prob LOW_RES_AUGMENTATION_PROB]
               [--crop_augmentation_prob CRO

In [13]:
# !NUM_CLASSES=$(find "${DATA_ROOT}/${DATASET_NAME}/imgs" -mindepth 1 -maxdepth 1 -type d | wc -l)
print(f"data_root:{data_root}, dataset_dir_name:{dataset_dir_name}")
!CUDA_VISIBLE_DEVICES=2 python AdaFace/main.py \
    --data_root {data_root} \
    --train_data_path {dataset_dir_name} \
    --val_data_path {validation_dir_name} \
    --prefix ir101_ms1mv2_adaface \
    # --use_mxrecord \
    --gpus 1 \
    --use_16bit \
    --arch ir_101 \
    --batch_size 64 \
    --num_workers 16 \
    --epochs 5 \
    --lr_milestones 12,20,24 \
    --lr 0.1 \
    --head adaface \
    --m 0.4 \
    --h 0.333 \
    --low_res_augmentation_prob 0.2 \
    --crop_augmentation_prob 0.2 \
    --photometric_augmentation_prob 0.2 

data_root:data, dataset_dir_name:debug_dataset
classnum: 5

\AdaFace with the following property
self.m 0.5
self.h 0.0
self.s 64.0
self.t_alpha 0.01
Global seed set to 42
/data2/tomoshi0514/anaconda3/envs/AdafaceTest/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:441: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
/data2/tomoshi0514/anaconda3/envs/AdafaceTest/lib/python3.12/site-packages/lightning_lite/plugins/environments/slurm.py:167: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python AdaFace/main.py --data_root data --train_data_path d ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: F